<a href="https://colab.research.google.com/github/mralamdari/NLP-Chiper/blob/main/NLP_Chiper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import re
import os
import string
import textwrap
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
string.ascii_lowercase #r'[a-z]'

'abcdefghijklmnopqrstuvwxyz'

In [4]:
string.ascii_uppercase #r'[A-Z]'

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [5]:
string.ascii_letters   #r'[a-zA-Z]'

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [11]:
letters1 = list(string.ascii_lowercase)
letters2 = list(string.ascii_lowercase)

In [12]:
np.random.shuffle(letters2)

In [13]:
mapping_dic = {}
for l1, l2 in zip(letters1, letters2):
  mapping_dic[l1] = l2

In [14]:
mapping_dic

{'a': 't',
 'b': 'x',
 'c': 'z',
 'd': 'b',
 'e': 'r',
 'f': 'h',
 'g': 'p',
 'h': 'l',
 'i': 'c',
 'j': 'm',
 'k': 'g',
 'l': 'u',
 'm': 'a',
 'n': 'o',
 'o': 'v',
 'p': 'd',
 'q': 'f',
 'r': 'n',
 's': 'j',
 't': 'e',
 'u': 'w',
 'v': 'i',
 'w': 's',
 'x': 'y',
 'y': 'q',
 'z': 'k'}

In [19]:
# Letters ==> [97-122]
for i in letters1:
  print(ord(i))

97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122


In [20]:
M = np.ones((26, 26))
pi = np.zeros(26)

def update_transition(ch1, ch2):
  i = ord(ch1) - 97
  j = ord(ch2) - 97
  M[i, j] += 1

In [39]:
def update_pi(ch):
  i = ord(ch) - 97
  pi[i] += 1

In [23]:
def word_prob(word):
  i = ord(word[0]) - 97
  log_p = np.log(pi[i])

  for ch in word[1:]: 
    j = ord(ch) - 97
    log_p += np.log(M[i, j])
    i=j
  return log_p

In [24]:
def sequence_prob(sentence):
  if type(sentence) == str:
    sentence = sentence.split()
  
  total_log_p = 0
  for word in sentence:
    total_log_p += word_prob(word)
  return total_log_p

In [35]:
# !wget 'https://lazyprogrammer.me/course_files/moby_dick.txt'

if not os.path.exists('moby_dick.txt'):
  r = requests.get('https://lazyprogrammer.me/course_files/moby_dick.txt')
  with open('moby_dick.txt', 'w') as f:
    f.write(r.content.decode())

In [36]:
regex = re.compile('[^a-zA-Z]')

In [40]:
for line in open('moby_dick.txt'):
  line = line.rstrip()
  if line:
    line = regex.sub(' ', line)
    tokens = line.lower().split()
    for token in tokens:
      ch0 = token[0]
      update_pi(ch0)

      for ch1  in token[1:]:
        update_transition(ch0, ch1)
        ch0 = ch1
  pi /= pi.sum()
  M /= M.sum(axis=1, keepdims=True)

In [41]:
pi

array([1.82617134e-01, 7.63873099e-05, 1.80555582e-01, 1.49525408e-02,
       5.89247993e-15, 1.66667106e-01, 5.45290621e-06, 2.78486395e-02,
       1.48867670e-02, 5.53517367e-26, 2.80799957e-36, 9.92067232e-04,
       1.48867909e-02, 1.98449039e-03, 3.33410397e-01, 9.97514390e-04,
       1.37218916e-84, 2.77777781e-02, 1.52411032e-02, 1.50339690e-02,
       9.97538629e-04, 1.19463184e-11, 1.06874133e-03, 0.00000000e+00,
       1.86689290e-27, 0.00000000e+00])

In [45]:
M[0]

array([0.00000000e+000, 3.58819384e-014, 1.11111213e-001, 1.73611272e-004,
       0.00000000e+000, 5.55555556e-002, 8.68055556e-004, 3.58819384e-014,
       7.84145930e-003, 0.00000000e+000, 8.68055556e-004, 5.88352784e-006,
       8.04115161e-010, 6.73676048e-001, 0.00000000e+000, 4.19097276e-015,
       0.00000000e+000, 6.97917673e-002, 1.40630787e-002, 6.34121657e-002,
       9.96720169e-016, 8.68055556e-004, 8.68055716e-004, 4.01877572e-009,
       8.96990741e-004, 1.61926312e-177])

In [51]:
pd.DataFrame(M.round(2), index=letters1, columns=letters1)

,a,b,c,d,e,f,g,h,i,j,...,q,r,s,t,u,v,w,x,y,z
a,0.00,0.0,0.11,0.00,0.00,0.06,0.00,0.00,0.01,0.0,...,0.0,0.07,0.01,0.06,0.00,0.00,0.00,0.0,0.00,0.0
b,0.00,0.0,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.00,0.00,0.11,0.00,0.00,0.0,0.38,0.0
c,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.70,0.10,0.0,...,0.0,0.10,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0
d,0.07,0.0,0.00,0.00,0.27,0.00,0.00,0.00,0.01,0.0,...,0.0,0.63,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0
e,0.06,0.0,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.47,0.00,0.04,0.00,0.04,0.01,0.0,0.00,0.0
f,0.00,0.0,0.00,0.00,0.02,0.02,0.00,0.00,0.02,0.0,...,0.0,0.00,0.00,0.33,0.00,0.00,0.00,0.0,0.00,0.0
g,0.01,0.0,0.00,0.00,0.58,0.00,0.00,0.09,0.00,0.0,...,0.0,0.04,0.00,0.03,0.00,0.00,0.00,0.0,0.00,0.0
h,0.31,0.0,0.00,0.00,0.44,0.00,0.00,0.00,0.25,0.0,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0
i,0.00,0.0,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.13,0.02,0.00,0.00,0.00,0.0,0.00,0.0
j,0.23,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.00,0.00,0.00,0.27,0.00,0.00,0.0,0.00,0.0


In [53]:
sample = '''My cabby drove fast. I don't think I ever drove faster, but the others
were there before us. The cab and the landau with their steaming horses
were in front of the door when I arrived. I paid the man and hurried
into the church. There was not a soul there save the two whom I had
followed and a surpliced clergyman, who seemed to be expostulating with
them. They were all three standing in a knot in front of the altar. I
lounged up the side aisle like any other idler who has dropped into a
church. Suddenly, to my surprise, the three at the altar faced round to
me, and Godfrey Norton came running as hard as he could towards me.
'''

In [89]:
def encoder(text):
  text = text.lower()
  text = regex.sub(' ', text)
  coded_text = []
  for char in text:
    maped_char = mapping_dic.get(char)
    maped_char = maped_char if maped_char else char
    coded_text.append(maped_char)
  return ''.join(coded_text)

In [90]:
encoded_sample = encoder(sample)

In [91]:
encoded_sample

'aq ztxxq bnvir htje  c bvo e elcog c rirn bnvir htjern  xwe elr velrnj srnr elrnr xrhvnr wj  elr ztx tob elr utobtw scel elrcn jertacop lvnjrj srnr co hnvoe vh elr bvvn slro c tnncirb  c dtcb elr ato tob lwnncrb coev elr zlwnzl  elrnr stj ove t jvwu elrnr jtir elr esv slva c ltb hvuuvsrb tob t jwnduczrb zurnpqato  slv jrrarb ev xr rydvjewutecop scel elra  elrq srnr tuu elnrr jetobcop co t gove co hnvoe vh elr tuetn  c uvwoprb wd elr jcbr tcjur ucgr toq velrn cburn slv ltj bnvddrb coev t zlwnzl  jwbbrouq  ev aq jwndncjr  elr elnrr te elr tuetn htzrb nvwob ev ar  tob pvbhnrq ovnevo ztar nwoocop tj ltnb tj lr zvwub evstnbj ar  '

In [107]:
decoder_dic = {}
for k, v in mapping_dic.items():
  decoder_dic[v] = k

In [108]:
decoder_dic

{'a': 'm',
 'b': 'd',
 'c': 'i',
 'd': 'p',
 'e': 't',
 'f': 'q',
 'g': 'k',
 'h': 'f',
 'i': 'v',
 'j': 's',
 'k': 'z',
 'l': 'h',
 'm': 'j',
 'n': 'r',
 'o': 'n',
 'p': 'g',
 'q': 'y',
 'r': 'e',
 's': 'w',
 't': 'a',
 'u': 'l',
 'v': 'o',
 'w': 'u',
 'x': 'b',
 'y': 'x',
 'z': 'c'}

In [111]:
def decoder(text):
  decoded_text = []
  for char in text:
    maped_char = decoder_dic.get(char)
    maped_char = maped_char if maped_char else char
    decoded_text.append(maped_char)
  return ''.join(decoded_text)

In [114]:
decoded_sample = decoder(encoded_sample)

In [115]:
decoded_sample

'my cabby drove fast  i don t think i ever drove faster  but the others were there before us  the cab and the landau with their steaming horses were in front of the door when i arrived  i paid the man and hurried into the church  there was not a soul there save the two whom i had followed and a surpliced clergyman  who seemed to be expostulating with them  they were all three standing in a knot in front of the altar  i lounged up the side aisle like any other idler who has dropped into a church  suddenly  to my surprise  the three at the altar faced round to me  and godfrey norton came running as hard as he could towards me  '

In [125]:
pred = np.array(decoded_sample.split())
original = np.array(regex.sub(' ', sample.lower()).split())

In [126]:
pred[:10]

array(['my', 'cabby', 'drove', 'fast', 'i', 'don', 't', 'think', 'i',
       'ever'], dtype='<U13')

In [127]:
original[:10]

array(['my', 'cabby', 'drove', 'fast', 'i', 'don', 't', 'think', 'i',
       'ever'], dtype='<U13')

In [128]:
pred.shape, original.shape

((124,), (124,))

In [130]:
100 * np.sum(pred == original) / len(pred)

100.0

In [151]:
dna_population = []
for _ in range(10):
  np.random.shuffle(letters2)
  dna_population.append(letters2)

In [153]:
dna_population[2]

['z',
 'i',
 'v',
 'h',
 'c',
 'n',
 'a',
 'd',
 'l',
 'e',
 'q',
 'g',
 'w',
 't',
 'o',
 's',
 'r',
 'j',
 'p',
 'm',
 'u',
 'y',
 'b',
 'f',
 'x',
 'k']

In [154]:
def offspring(pop, num_children):
  offspring_list=[]
  for dna in pop:
    for i in range(num_children):
      copy_dna = dna.copy()
      j = np.random.randint(len(copy_dna))
      k = np.random.randint(len(copy_dna))

      temp = copy_dna[j]
      copy_dna[j] = copy_dna[k]
      copy_dna[k] = temp
      offspring_list.append(copy_dna)
  return offspring_list + pop

In [ ]:
num_iters = 100
scores = np.zeros(num_iters)
best_dna, best_map, best_score = None, None, float('-inf')
for i in range(num_iters):
  if i > 0:
    dna_population = offspring(dna_population, 2)

  dna2scores = {}
  for dna in dna_population:
    decoded_text = decoder(encoded_sample)
    score = sequence_prob(decoded_text)
    dna2scores[''.join(dna)] = score

    if score > best_score:
      best_score = score
      best_dna = dna
      best_map = encoded_sample
  
  scores[i] = np.mean(list(dna2scores.values()))
  sorted_dna = sorted(dna2scores.items(), key=lambda x: x[1], reverse=True)
  dna_pool = [list(k) for k, v in sorted_dna[:5]]

  if i%10 == 0:
    print(f'Iteration {i}, Current score is: {scores[i]}, Best Score is: {best_score}')

Iteration 0, Current score is: -4120.423628346766, Best Score is: -4120.423628346766
